# Build a LLM-powered Chatbot in LangGraph

LangGraph is not just a framework to create static graphs. We already know that it can be used for building stateful, agentic applications using LLMs.

We'll now create a simple LLM-powered chatbot using LangGraph. This chatbot will respond directly to user messages.

![](https://i.imgur.com/heeggTe.png)

In [0]:
!pip install langchain==0.3.14
!pip install langgraph==0.2.66
!pip install langchain-openai==0.3.2

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph.

The State schema serves as the input schema for all Nodes and Edges in the graph.

Let's use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [0]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

## Create the Nodes, Edges and Graph

In [0]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [0]:
graph

In [0]:
response = graph.invoke({"messages": "Explain AI in 2 bullet points"})
response

In [0]:
response['messages']

In [0]:
print(response['messages'][-1].content)

## Invoking vs. Streaming in LangGraph

In [0]:
response = graph.invoke({"messages": "Explain AI in 1 line to a child"})
print(response['messages'][-1].content)

In [0]:
response = graph.invoke({"messages": "What did we discuss so far?"})
print(response['messages'][-1].content)

In [0]:
for event in graph.stream({"messages": "Explain AI in 1 line to a child"},
                          stream_mode='values'):
    print(event['messages'])

In [0]:
for event in graph.stream({"messages": "Explain AI in 1 line to a child"},
                          stream_mode='updates'):
    print(event)